In [1]:
!pip install --upgrade transformers torch accelerate bitsandbytes unsloth --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 91.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 14.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.0 MB/s eta 0:00:

In [2]:
import torch
from transformers import pipeline
import pandas as pd
import json
import gc

2025-08-11 03:09:41.989925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754881782.318663      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754881782.416057      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def initialize_model():
    model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"
    print(f"--- Initializing model: {model_id} ---")
    
    try:
        llm_pipeline = pipeline(
            "text-generation",
            model=model_id,
            model_kwargs={"torch_dtype": "auto"},
            device_map="auto",
        )
        print("--- Model initialized successfully. ---")
        return llm_pipeline
    except Exception as e:
        print(f"Error initializing model: {e}")
        return None

In [4]:
def generate_response(llm_pipeline, prompt: str) -> str:
    messages = [{"role": "user", "content": prompt}]
    
    try:
        outputs = llm_pipeline(
            messages,
            max_new_tokens=1024,
            eos_token_id=llm_pipeline.tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.1,
            top_p=0.9
        )
        response_text = outputs[0]['generated_text'][-1]['content']
        return response_text
    except Exception as e:
        print(f"An error occurred during model inference: {e}")
        return "Error: Could not generate response."

In [5]:
IDIOMATIC_EXPRESSIONS = {
    "raining cats and dogs",
    "costs an arm and a leg",
    "bite the bullet",
    "let the cat out of the bag",
    "feeling under the weather",
    "hit the nail on the head",
    "piece of cake",
    "apple of my eye",
    "heart of gold",
    "cross that bridge when we get there",
    "butterflies in my stomach",
    "you've got some nerve",
    "survival of the fittest",
    "tip of the iceberg"
}

def is_idiom(phrase: str) -> bool:
    is_found = phrase.lower().strip() in IDIOMATIC_EXPRESSIONS
    print(f"Tool Used: is_idiom('{phrase}') -> Result: {is_found}")
    return is_found

WORD_CONNOTATIONS = {
    "yaman": "formal",
    "salaysay": "formal",
    "datapwat": "formal",
    "kinikilala": "formal",
    "nagtataglay": "formal",
    "magapi": "formal",
    "wasakin": "formal",
    "mamumuno": "formal",
    "isinusulat": "formal",
    "kaganapan": "formal",

    "pera": "informal/colloquial",
    "talaga": "informal/emphatic",
    "sobrang": "informal",
    "barkada": "informal",
    "pinag-trip-an": "slang",
    "sabi": "colloquial",
    "pre": "slang",
    "badtrip": "colloquial/borrowed",
    "astig": "colloquial",
    "mukha mo": "colloquial/insult",

    "single": "colloquial/borrowed",
    "check-up": "colloquial/borrowed",
    "ma-admit": "colloquial/borrowed",
    "buzz-kill": "colloquial/borrowed",
    "probono": "technical/borrowed"
}

def get_word_connotation(term: str) -> str:
    """
    Checks the connotation of a specific English or Filipino word.
    Returns 'formal', 'informal', 'slang', 'neutral', or 'unknown'.
    """
    term_lower = term.lower().strip(".,?!")
    connotation = WORD_CONNOTATIONS.get(term_lower, "neutral/unknown")
    print(f"Tool Used: get_term_connotation('{term}') -> Result: {connotation}")
    return f"The term '{term}' has a connotation of: {connotation}."


TECHNICAL_JARGON = {
    "stack overflow": "Tech",
    "call stack": "Tech",
    "database index": "Tech",
    "digital signature": "Tech",
    "time complexity": "Tech",
    "binary search": "Tech",
    "multithreading": "Tech",
    "recursion": "Tech",
    "hash function": "Tech",
    "big o notation": "Tech",
    "race condition": "Tech",
    "virtual memory": "Tech",
    "sql": "Tech",
    "os scheduler": "Tech",
    "linked lists": "Tech",
    "public key cryptography": "Tech",
    "logic gate": "Tech",

    "polytrauma": "Medical",
    "purulent discharge": "Medical",
    "refractory to medication": "Medical",
    "nightmare disorder": "Medical",
    "body dysmorphic disorder": "Medical",
    "suicidal ideation": "Medical",
    "cognitive distortions": "Medical",
    "empty stomach": "Medical",

    "oshia violations": "Gaming/Cultural Reference",
    "health violations": "Gaming",
    "sentient cardiovascular system": "Gaming/Joke",
    "telepathic dog": "Gaming/Joke",
    "get owned": "Gaming Slang",
    "buzz-kill": "Gaming/Slang",

    "party of the first part": "Legal",
    "probono": "Legal"
}

def is_technical_jargon(term: str) -> str:
    """
    Checks if a term is established jargon from a specific domain.
    This tells the agent that the term should likely be preserved, not translated literally.
    """
    term_lower = term.lower().strip()
    domain = TECHNICAL_JARGON.get(term_lower)
    if domain:
        result = f"'{term}' is an established jargon in the '{domain}' domain."
        print(f"Tool Used: is_technical_jargon('{term}') -> Result: {result}")
        return result
    else:
        result = f"'{term}' is not found in the technical jargon list."
        print(f"Tool Used: is_technical_jargon('{term}') -> Result: {result}")
        return result

In [6]:
import json

class AgenticJudge:
    def __init__(self):
        self.llm_pipeline = initialize_model()
        self.memory = {} 

    def run_evaluation(self, source_text: str, translation_text: str):
        if not self.llm_pipeline:
            print("Evaluation cannot proceed; model not initialized.")
            return

        print("\n\n--- STARTING AGENTIC EVALUATION ---")
        self.memory['source_text'] = source_text
        self.memory['translation_text'] = translation_text

        self._step_1_accuracy_check()
        self._step_2_tone_analysis()
        self._step_final_synthesis()
        
        print("--- AGENTIC EVALUATION COMPLETE ---")
        return self.memory.get('final_evaluation')

    def _execute_step(self, step_name: str, prompt: str):
        """A helper function to run a step and store its result in memory."""
        print(f"\n--- Agent Thought: Executing {step_name} ---")
        response = generate_response(self.llm_pipeline, prompt)
        self.memory[step_name] = response
        print(f"--- Agent Observation ({step_name}): ---")
        print(response)

    def _step_1_accuracy_check(self):
        prompt = f"""
        Source Text: "{self.memory['source_text']}"
        Translation: "{self.memory['translation_text']}"
        
        Task: First, focus ONLY on literal accuracy. Does the translation convey the same basic information as the source? Ignore tone, style, and fluency for now. Provide a brief, one-paragraph analysis.
        """
        self._execute_step("accuracy_analysis", prompt)

    def _step_2_tone_analysis(self):
        # This prompt builds on the previous step's output (memory)
        prompt = f"""
        Previous Analysis (Accuracy): "{self.memory['accuracy_analysis']}"
        Source Text: "{self.memory['source_text']}"
        Translation: "{self.memory['translation_text']}"
    
        Task: Now, analyze Tone & Style. Follow these sub-steps carefully:
        1.  First, describe the tone of the English source text on its own. Is it formal, informal, poetic, clinical, etc.? Justify your answer.
        2.  Second, describe the tone of the Filipino translation on its own.
        3.  Third, compare the two tones. Do they match?
        4.  Finally, identify any specific Filipino words whose connotation you want to verify. If you find a word, output it in the format: [TOOL_CALL: get_term_connotation('word')]
        """
        # Thought -> Action (potential tool use) -> Observation
        print(f"\n--- Agent Thought: Executing Tone Analysis ---")
        response = generate_response(self.llm_pipeline, prompt)
        print(f"--- Agent Observation (Tone Analysis - Raw): ---\n{response}")

        # Check if the LLM decided to use a tool
        if "[TOOL_CALL: get_word_connotation" in response:
            # Simple parsing for the tool call
            try:
                word_to_check = response.split("('")[1].split("')")[0]
                tool_result = get_word_connotation(word_to_check)
                
                # Now, ask the agent to reflect on the tool's output
                reflection_prompt = f"""
                Your initial tone analysis was: "{response}"
                You decided to use a tool to check the word '{word_to_check}', and the tool returned this information: "{tool_result}"
                
                Task: Based on this new information, please provide your final, updated analysis of the Tone & Style.
                """
                self._execute_step("tone_analysis_final", reflection_prompt)
            except Exception as e:
                print(f"Error parsing or using tool: {e}")
                self.memory["tone_analysis_final"] = response # Fallback
        else:
            self.memory["tone_analysis_final"] = response # No tool was used

    def _step_final_synthesis(self):
        initial_prompt = f"""
        You are a meticulous judge creating a final evaluation. You have performed a step-by-step analysis.
        Here is your entire scratchpad of notes:
        - Accuracy Analysis: {self.memory.get('accuracy_analysis')}
        - Tone & Style Analysis: {self.memory.get('tone_analysis_final')}
        
        Final Judgment Guide:
        Your first task is to decide which of the following categories your 'Tone & Style Analysis' fits into. Then, assign the corresponding score.

        Tone & Style Scoring Guide:
        - (Score: 5): The tones are a perfect or near-perfect match.
        - (Score: 3-4): The tones are broadly similar but have minor differences.
        -  (Score: 1-2): The tones are clearly mismatched and fundamentally different.

        Your Task:
        Based on your notes and the guide above, provide the final evaluation.
        Adhere strictly to this JSON format and nothing else:
        {{
          "scores": {{
            "accuracy": <score from 1-5>,
            "tone_and_style": <score from 1-5, based on the Category you choose>,
            "nuance_and_metaphor": <score from 1-5>,
            "fluency": <score from 1-5>,
            "overall": <average_score>
          }},
          "explanation": "<Your detailed justification here. Start by explicitly stating which Category you chose and why your analysis supports this choice.>"
        }}
        """
        
        print(f"\n--- Agent Thought: Synthesizing Final Judgment (Attempt 1) ---")
        raw_output = generate_response(self.llm_pipeline, initial_prompt)
        
        try:
            start = raw_output.find('{')
            end = raw_output.rfind('}') + 1
            json_str = raw_output[start:end]
            self.memory['final_evaluation'] = json.loads(json_str)
            print(f"--- Agent Observation (Final Evaluation - Success): ---\n{self.memory['final_evaluation']}")
            return # Success, exit the function
        except json.JSONDecodeError as e:
            print(f"--- Agent Observation: Initial JSON parsing failed. Error: {e} ---")
            print("--- Agent Thought: Attempting self-correction. ---")
            
            correction_prompt = f"""
            The following text you generated resulted in a JSON parsing error.
            
            Original Text:
            ---
            {raw_output}
            ---

            The error message was:
            ---
            {e}
            ---

            Your Task: Please correct the JSON syntax and provide ONLY the valid, corrected JSON object. Do not include any other text or explanation.
            """
            
            corrected_output = generate_response(self.llm_pipeline, correction_prompt)
            
            try:
                start = corrected_output.find('{')
                end = corrected_output.rfind('}') + 1
                json_str = corrected_output[start:end]
                self.memory['final_evaluation'] = json.loads(json_str)
                print(f"--- Agent Observation (Self-Correction - Success): ---\n{self.memory['final_evaluation']}")
            except Exception as final_e:
                print(f"--- Agent Observation: Self-correction also failed. Error: {final_e} ---")
                self.memory['final_evaluation'] = {"error": "Failed to generate valid JSON even after self-correction.", "final_attempt_output": corrected_output}




In [11]:
# def main():
#     # --- Configuration ---
#     INPUT_CSV = '/kaggle/input/testdraft/test.csv'
#     OUTPUT_JSON = 'agentic_test_results.json'

#     # Initialize the agent, which handles its own model.
#     judge = AgenticJudge()
#     if not judge.llm_pipeline:
#         print("Aborting due to model initialization failure.")
#         return

#     # Load the test dataset.
#     try:
#         df = pd.read_csv(INPUT_CSV)
#         df.rename(columns={df.columns[0]: "English"}, inplace=True)
#     except FileNotFoundError:
#         print(f"Error: Input file not found at '{INPUT_CSV}'.")
#         return

#     print(f"\n--- Running Agentic Judge on {len(df)} entries from {INPUT_CSV} ---")
    
#     all_results = []
#     for index, row in df.iterrows():
#         source = row.get('English')
#         translation = row.get('Filipino')

#         if pd.isna(source) or pd.isna(translation):
#             continue

#         print(f"\n{'='*20} Processing Entry {index + 1}/{len(df)} {'='*20}")
#         result = judge.run_evaluation(source, translation)

#         all_results.append({
#             "entry_index": index,
#             "source_text": source,
#             "target_text": translation,
#             "evaluation_result": result
#         })

#     # Save all collected results to a single JSON file.
#     with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
#         json.dump(all_results, f, ensure_ascii=False, indent=2)
        
#     print(f"\n--- Evaluation complete. Results saved to {OUTPUT_JSON} ---")


# if __name__ == "__main__":
#     main()

--- Initializing model: unsloth/llama-3-8b-Instruct-bnb-4bit ---


Device set to use cuda:0


--- Model initialized successfully. ---

--- Running Agentic Judge on 64 entries from /kaggle/input/valset/Datasets - Human-Labeled Validation Set.csv ---

--- Evaluation complete. Results saved to agentic_test_results.json ---


In [7]:
# In agentic_judge.py

def main():

    # --- Configuration ---
    INPUT_CSV = '/kaggle/input/valset/Datasets - Human-Labeled Validation Set.csv'
    OUTPUT_JSON = 'agentic_variance_subset_results.json'
    SUBSET_LIMIT = 5  # <-- How many unique items to test
    NUM_RUNS = 3      # <-- How many times to test each item

    # Initialize the agent.
    judge = AgenticJudge()
    if not judge.llm_pipeline:
        print("Aborting due to model initialization failure.")
        return

    # Load the validation dataset.
    try:
        df = pd.read_csv(INPUT_CSV)
    except FileNotFoundError:
        print(f"Error: Input file not found at '{INPUT_CSV}'.")
        return

    # Limit the dataframe to the specified subset
    df = df.head(SUBSET_LIMIT)

    print(f"\n--- Running Agentic Judge Variance Test ---")
    print(f"--- Testing {NUM_RUNS} runs on the first {len(df)} entries from {INPUT_CSV} ---")
    
    all_results = []
    # Outer loop: Iterate through each of the 5 items in the subset
    for index, row in df.iterrows():
        source = row.get('Source Text (English)')
        translation = row.get('Target Text (Filipino)')

        if pd.isna(source) or pd.isna(translation):
            continue

        print(f"\n{'='*20} Processing Entry {index + 1}/{len(df)} {'='*20}")
        
        variance_runs_for_this_entry = []
        # Inner loop: Run the evaluation NUM_RUNS times for this single item
        for i in range(NUM_RUNS):
            print(f"--> Run {i + 1}/{NUM_RUNS}...")
            result = judge.run_evaluation(source, translation)
            
            variance_runs_for_this_entry.append({
                "run_number": i + 1,
                "evaluation_result": result
            })
        
        # Append the collected variance runs for this entry to the main results list
        all_results.append({
            "entry_index": index,
            "source_text": source,
            "target_text": translation,
            "human_score": row.get("Final Score                          (1 - lowest, 5 - highest)"),
            "variance_runs": variance_runs_for_this_entry
        })

    # Save all collected results to a single JSON file.
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
        
    print(f"\n--- Variance test complete. Results saved to {OUTPUT_JSON} ---")

if __name__ == "__main__":
    main()

--- Initializing model: unsloth/llama-3-8b-Instruct-bnb-4bit ---


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Device set to use cuda:0


--- Model initialized successfully. ---

--- Running Agentic Judge Variance Test ---
--- Testing 3 runs on the first 5 entries from /kaggle/input/valset/Datasets - Human-Labeled Validation Set.csv ---

==================== Processing Entry 1/5 ====================
--> Run 1/3...


--- STARTING AGENTIC EVALUATION ---

--- Agent Thought: Executing accuracy_analysis ---
--- Agent Observation (accuracy_analysis): ---
The translation "Ang mga bata ay nagtawanan at naglaro sa ilalim ng hapon na araw" conveys the same basic information as the source text "The children laughed and played under the afternoon sun". The translation accurately captures the main elements of the source text, including the subject (children), the actions (laughed and played), and the location (under the afternoon sun). The use of the Filipino phrase "ilalim ng hapon na araw" to describe the location is a literal translation of the English phrase, and the verb phrases "nagtawanan" and "naglaro" accurately convey the a

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


--- Agent Observation (accuracy_analysis): ---
The translation "Nagpahinga siya para mag-isip-isip" conveys a different meaning than the source text "She took a break to gather her thoughts." While both sentences mention taking a break, the Filipino translation implies that the person is resting or sleeping ("pahinga" means to rest or sleep), whereas the English source text suggests a more general break or pause to collect one's thoughts. The Filipino translation does not accurately convey the idea of gathering thoughts, which is a key aspect of the original sentence.

--- Agent Thought: Executing Tone Analysis ---
--- Agent Observation (Tone Analysis - Raw): ---
**Tone & Style Analysis**

**Step 1: English Source Text Tone**

The English source text "She took a break to gather her thoughts" has a neutral, informal tone. The language is straightforward and simple, conveying a sense of everyday conversation. The tone is not formal or clinical, but rather casual and conversational. The u

In [14]:
# def main():

#     INPUT_CSV = '/kaggle/input/valset/Datasets - Human-Labeled Validation Set.csv'
#     OUTPUT_JSON = 'agentic_validation_results.json'

#     # Initialize the agent, which handles its own model.
#     judge = AgenticJudge()
#     if not judge.llm_pipeline:
#         print("Aborting due to model initialization failure.")
#         return

#     # Load the validation dataset.
#     try:
#         df = pd.read_csv(INPUT_CSV)
#         # --- A good debugging step is to always print the column headers ---
#         print("CSV Columns Found:", df.columns.tolist())
#     except FileNotFoundError:
#         print(f"Error: Input file not found at '{INPUT_CSV}'.")
#         return

#     print(f"\n--- Running Agentic Judge on {len(df)} entries from {INPUT_CSV} ---")
    
#     all_results = []
#     for index, row in df.iterrows():
#         # --- THIS IS THE CRITICAL FIX ---
#         # Use the correct, full column names from your validation CSV
#         source = row.get('Source Text (English)')
#         translation = row.get('Target Text (Filipino)')
#         # ---------------------------------

#         if pd.isna(source) or pd.isna(translation):
#             # This is the line that was being triggered for every row before the fix
#             print(f"Skipping row {index + 1} due to missing source or translation text.")
#             continue

#         print(f"\n{'='*20} Processing Entry {index + 1}/{len(df)} {'='*20}")
#         result = judge.run_evaluation(source, translation)

#         all_results.append({
#             "entry_index": index,
#             "source_text": source,
#             "target_text": translation,
#             "evaluation_result": result,
#             "human_score": row.get("Final Score                          (1 - lowest, 5 - highest)"), # Also grab the human score for comparison
#             "human_explanation": row.get("Rater 1 Explanation")
#         })

#     # Save all collected results to a single JSON file.
#     with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
#         json.dump(all_results, f, ensure_ascii=False, indent=2)
        
#     print(f"\n--- Evaluation complete. Results saved to {OUTPUT_JSON} ---")


# if __name__ == "__main__":
#     main()

--- Initializing model: unsloth/llama-3-8b-Instruct-bnb-4bit ---


Device set to use cuda:0


--- Model initialized successfully. ---
CSV Columns Found: ['Source Text (English)', 'Target Text (Filipino)', 'Final Score                          (1 - lowest, 5 - highest)', 'Rater 1 Explanation', 'Rater 2 Explanation', 'Contributor']

--- Running Agentic Judge on 64 entries from /kaggle/input/valset/Datasets - Human-Labeled Validation Set.csv ---

==================== Processing Entry 1/64 ====================


--- STARTING AGENTIC EVALUATION ---

--- Agent Thought: Executing accuracy_analysis ---
--- Agent Observation (accuracy_analysis): ---
The translation "Ang mga bata ay nagtawanan at naglaro sa ilalim ng hapon na araw" conveys the same basic information as the source text "The children laughed and played under the afternoon sun". The translation accurately captures the main elements of the source text, including the subject (children), the actions (laughed and played), and the location (under the afternoon sun). The translation also uses the correct Filipino phrases to conve

In [41]:
# def main():
#     judge = AgenticJudge()
    
#     source = "It's raining cats and dogs."
#     translation = "Umuulan ng pusa at aso."

#     result = judge.run_evaluation(source, translation)

#     if result:
#         print("\n\n--- FINAL AGENTIC JUDGE OUTPUT ---")
#         print(json.dumps(result, indent=2))

# if __name__ == "__main__":
#     main()

--- Initializing model: unsloth/llama-3-8b-Instruct-bnb-4bit ---


Falling back to torch.float32 because loading with the original dtype failed on the target device.
Device set to use cuda:1


--- Model initialized successfully. ---


--- STARTING AGENTIC EVALUATION ---

--- Agent Thought: Executing accuracy_analysis ---
--- Agent Observation (accuracy_analysis): ---
A literal analysis of the translation reveals that it does not convey the same basic information as the source text. The phrase "It's raining cats and dogs" is an idiomatic expression that means raining heavily, whereas the translation "Umuulan ng pusa at aso" is a literal translation that means "it's raining cats and dogs" in a very literal sense. The translation does not capture the figurative meaning of the original phrase, instead providing a nonsensical and humorous interpretation.

--- Agent Thought: Executing Tone Analysis ---
--- Agent Observation (Tone Analysis - Raw): ---
Here's the analysis of Tone & Style:

**Step 1: Describe the tone of the English source text**

The tone of the English source text "It's raining cats and dogs" is informal, colloquial, and idiomatic. The phrase is a common expression

In [7]:
# def main():
#     judge = AgenticJudge()
    
#     source = "Fear is the mind-killer. Fear is the little-death that brings total obliteration."
#     translation =  "Ang takot ay nakakamatay ng isip. Ang takot ay parang maliit na kamatayan na nagdadala ng pagkasira."

#     result = judge.run_evaluation(source, translation)

#     if result:
#         print("\n\n--- FINAL AGENTIC JUDGE OUTPUT ---")
#         print(json.dumps(result, indent=2))

# if __name__ == "__main__":
#     main()

--- Initializing model: unsloth/llama-3-8b-Instruct-bnb-4bit ---


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Device set to use cuda:0


--- Model initialized successfully. ---


--- STARTING AGENTIC EVALUATION ---

--- Agent Thought: Executing accuracy_analysis ---
--- Agent Observation (accuracy_analysis): ---
The translation provided conveys the same basic information as the source text in terms of literal accuracy. The translation accurately captures the two main ideas presented in the source text: "Fear is the mind-killer" and "Fear is the little-death that brings total obliteration". The translation uses the same key phrases, "Fear is", "mind-killer", and "little-death", to convey the same meaning as the source text. The translation also maintains the same structure and sentence organization as the source text, which helps to ensure that the same information is being conveyed. However, it's worth noting that the translation may not fully capture the nuances of the original text, such as the tone, style, and connotation of the words used.

--- Agent Thought: Executing Tone Analysis ---
--- Agent Observation (Tone A